## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP2 - Preparación de los datos

---

En esta libreta se realizan las transforamciones sobre los datasets del escenario en función 
de los resultados de la verificación de calidad de datos. 

---

In [1]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

In [2]:
df_creditos = pd.read_csv("../../data/processed/datos_creditos_mc.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../data/processed/datos_tarjetas_mc.csv", sep=";")
display(df_tarjetas.head(1))

df_integrado = pd.read_csv("../../data/processed/datos_integrados.csv", sep=";")
display(df_integrado.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,0,N,0.0,True,True


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,estado_civil_ok,estado_cliente_ok
0,708082083.0,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,Blue,111.0,3.0,True,True


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,estado_civil_ok,estado_cliente_ok,regla_pct_duracion_situacion
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,...,15149.0,F,3544.0,SECUNDARIO_COMPLETO,Blue,111.0,3.0,True,True,ok


---
#### Aplicación de transformaciones

**Operaciones a realizar**

1. Selección de columnas
2. Filtrado de filas
3. Construcción de atributos
4. Integración de datasets
5. Formateo definitivo


----

Selección de datos

In [3]:
# Se establece qué columnas se eliminan

col_eliminar_creditos = ['completitud_fila', 'situacion_vivienda_ok', 'objetivo_credito_ok', 'tasa_interes', 'antiguedad_empleado']
col_eliminar_tarjetas = ['nivel_tarjeta', 'estado_civil_ok', 'estado_cliente_ok']

# Se ejecuta la operación

df_creditos.drop(col_eliminar_creditos, inplace=True, axis=1)
df_tarjetas.drop(col_eliminar_tarjetas, inplace=True, axis=1)

In [4]:
print("Vista del dataset de datos de créditos:")
display(df_creditos.head(1))

print("Vista del dataset de datos de tarjetas:")
display(df_tarjetas.head(1))

Vista del dataset de datos de créditos:


,id_cliente,edad,importe_solicitado,duracion_credito,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,estado_credito,falta_pago
0,708082083.0,24,11000,3,HIPOTECA,64800,INVERSIONES,0.17,0,N


Vista del dataset de datos de tarjetas:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,operaciones_ult_12m,personas_a_cargo
0,708082083.0,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,111.0,3.0


Limpieza de los datos (filtrado a nivel de filas)

*Dataset créditos*

Regla 1

In [5]:
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_creditos.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_creditos = pd.concat([df_creditos, regla_pct_ingresos], axis=1)
df_creditos.head(5)  

,id_cliente,edad,importe_solicitado,duracion_credito,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,estado_credito,falta_pago,regla_pct_ingresos
0,708082083.0,24,11000,3,HIPOTECA,64800,INVERSIONES,0.17,0,N,ok
1,708083283.0,24,1500,2,ALQUILER,30996,MEJORAS_HOGAR,0.05,1,Y,ok
2,708084558.0,23,10000,2,OTROS,40104,EDUCACIÓN,0.25,0,Y,ok
3,708085458.0,25,6000,4,ALQUILER,23198,INVERSIONES,0.26,0,N,ok
4,708086958.0,26,10000,2,HIPOTECA,50000,EDUCACIÓN,0.20,0,N,ok


In [6]:
df_creditos[df_creditos['regla_pct_ingresos'] == 'err'].shape[0]

15

Regla 2

In [7]:
def regla_2(row):
    pct_ingreso = row.pct_ingreso
    duracion = row.duracion_credito
    situacion_vivienda = row.situacion_vivienda
    
    if duracion == 2 and pct_ingreso > 0.6 and situacion_vivienda != 'PROPIA':
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'

# Se aplica la función para todos los elementos del dataset
regla_2 = df_creditos.apply(lambda row: regla_2(row), axis=1).rename("regla_2")

# Se unen los resultados al dataset inicial
df_creditos = pd.concat([df_creditos, regla_2], axis=1)
df_creditos.head(5)  

,id_cliente,edad,importe_solicitado,duracion_credito,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,estado_credito,falta_pago,regla_pct_ingresos,regla_2
0,708082083.0,24,11000,3,HIPOTECA,64800,INVERSIONES,0.17,0,N,ok,ok
1,708083283.0,24,1500,2,ALQUILER,30996,MEJORAS_HOGAR,0.05,1,Y,ok,ok
2,708084558.0,23,10000,2,OTROS,40104,EDUCACIÓN,0.25,0,Y,ok,ok
3,708085458.0,25,6000,4,ALQUILER,23198,INVERSIONES,0.26,0,N,ok,ok
4,708086958.0,26,10000,2,HIPOTECA,50000,EDUCACIÓN,0.20,0,N,ok,ok


In [8]:
df_creditos[df_creditos['regla_2'] == 'err'].shape[0]

7

In [9]:
# Se filtran las filas con algún error detectado
print(f"Filas antes del filtro: {df_creditos.shape[0]}")

temp = df_creditos[df_creditos['edad'] < 90]

# Otro filtro posible: por la regla de negocio agregada

temp_c = temp[temp['regla_pct_ingresos'] == 'ok']
temp_cc = temp_c[temp_c['regla_2'] == 'ok']

print(f"Filas después del filtro: {temp_cc.shape[0]}")

Filas antes del filtro: 10127
Filas después del filtro: 10103


*Dataset tarjetas*

In [12]:
# Se filtran las filas con algún error detectado
print(f"Filas antes del filtro: {df_tarjetas.shape[0]}")

temp_t = df_tarjetas[df_tarjetas['limite_credito_tc'] != 34516]

print(f"Filas después del filtro: {temp_t.shape[0]}")

print(f"Filas filtradas: {df_tarjetas.shape[0] - temp_t.shape[0]}")

Filas antes del filtro: 10127
Filas después del filtro: 9619
Filas filtradas: 508


Integración de datos

In [13]:
df_integrado = pd.merge(temp_cc, temp_t, on='id_cliente', how='inner')
coincidencias = df_integrado.shape[0]

print(f"Filas del dataset integrado con los filtros realizados: {coincidencias}")

Filas del dataset integrado con los filtros realizados: 9595


In [14]:
print(f"Cantidad de columnas del dataset integrado: {df_integrado.shape[1]}")

Cantidad de columnas del dataset integrado: 21


#### Transformación de atributos

Atributos nominales que se modifican los valores

In [15]:
# Columna: estado_civil
cambios_estado_civil = {
    'CASADO' : 'C',
    'SOLTERO' : 'S',
    'DESCONOCIDO' : 'N',
    'DIVORCIADO' : 'D',
}

estado_civil_N = df_integrado.loc[:, ('estado_civil')].map(cambios_estado_civil).rename('estado_civil_N')

# Columna: estado_credito
cambios_estado_credito = {
    0: 'C',
    1 : 'P',
}

estado_credito_N = df_integrado.loc[:, ('estado_credito')].map(cambios_estado_credito).rename('estado_credito_N')

df_integrado = pd.concat([estado_civil_N, estado_credito_N, df_integrado], axis=1)
df_integrado.head()

# Sobre este resultado será necesario eliminar las columnas auxiliares

,estado_civil_N,estado_credito_N,id_cliente,edad,importe_solicitado,duracion_credito,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,...,regla_2,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,operaciones_ult_12m,personas_a_cargo
0,C,C,708082083.0,24,11000,3,HIPOTECA,64800,INVERSIONES,0.17,...,ok,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,111.0,3.0
1,S,P,708083283.0,24,1500,2,ALQUILER,30996,MEJORAS_HOGAR,0.05,...,ok,45.0,SOLTERO,PASIVO,992.0,M,3421.0,DESCONOCIDO,21.0,0.0
2,D,C,708084558.0,23,10000,2,OTROS,40104,EDUCACIÓN,0.25,...,ok,38.0,DIVORCIADO,PASIVO,1447.0,M,8258.0,POSGRADO_COMPLETO,23.0,3.0
3,S,C,708085458.0,25,6000,4,ALQUILER,23198,INVERSIONES,0.26,...,ok,24.0,SOLTERO,ACTIVO,3940.0,F,1438.3,UNIVERSITARIO_INCOMPLETO,82.0,2.0
4,C,C,708086958.0,26,10000,2,HIPOTECA,50000,EDUCACIÓN,0.20,...,ok,41.0,CASADO,ACTIVO,4369.0,F,3128.0,UNIVERSITARIO_INCOMPLETO,59.0,2.0


Atributos numéricos que se discretizan

In [17]:
df_integrado['edad'].describe()

count    9595.000000
mean       23.520271
std         1.512880
min        20.000000
25%        22.000000
50%        23.000000
75%        25.000000
max        26.000000
Name: edad, dtype: float64

In [18]:
# edad

etiquetas_e = ['menor_25', '25_a_30']
rangos_e = [0, 24, 50]
edad_N = pd.cut(df_integrado['edad'], 
                                bins=rangos_e, 
                                labels=etiquetas_e)

edad_N.value_counts()

edad
menor_25    6802
25_a_30     2793
Name: count, dtype: int64

In [19]:
df_integrado['ingresos'].describe()

count      9595.000000
mean      50087.002084
std       28183.523250
min        9600.000000
25%       33600.000000
50%       46000.000000
75%       59000.000000
max      500000.000000
Name: ingresos, dtype: float64

In [20]:
# ingresos

etiquetas_i = ['hasta_20k', '20k_a_50k', '50k_a_100k', 'mayor_100k']
rangos_i = [0, 19999, 49999, 99999, 999999]
ingresos_N = pd.cut(df_integrado['ingresos'], 
                                bins=rangos_i, 
                                labels=etiquetas_i)

ingresos_N.value_counts()

ingresos
20k_a_50k     4959
50k_a_100k    3757
hasta_20k      451
mayor_100k     428
Name: count, dtype: int64

In [21]:
df_integrado['pct_ingreso'].describe()

count    9595.000000
mean        0.176271
std         0.108138
min         0.010000
25%         0.100000
50%         0.150000
75%         0.240000
max         0.710000
Name: pct_ingreso, dtype: float64

In [27]:
# pct_ingreso

etiquetas_p_i = ['hasta_20', '20_a_40', '40_a_60', 'mayor_60']
rangos_p_i = [0, 0.19, 0.39, 0.59, 0.99]
pct_ingreso_N = pd.cut(df_integrado['pct_ingreso'], 
                                bins=rangos_p_i, 
                                labels=etiquetas_p_i)

pct_ingreso_N.value_counts()

pct_ingreso
hasta_20    6130
20_a_40     3018
40_a_60      432
mayor_60      15
Name: count, dtype: int64

In [23]:
df_integrado['antiguedad_cliente'].describe()

count    9595.000000
mean       35.914747
std         8.056776
min        13.000000
25%        31.000000
50%        36.000000
75%        40.000000
max        56.000000
Name: antiguedad_cliente, dtype: float64

In [24]:
# antiguedad_cliente

etiquetas_a_c = ['menor_2y', '2y_a_4y', 'mayor_4y']
rangos_a_c = [0, 24, 48, 100]
antiguedad_cliente_N = pd.cut(df_integrado['antiguedad_cliente'], 
                                bins=rangos_a_c, 
                                labels=etiquetas_a_c)

antiguedad_cliente_N.value_counts()

antiguedad_cliente
2y_a_4y     8143
menor_2y     818
mayor_4y     634
Name: count, dtype: int64

In [25]:
df_integrado['gastos_ult_12m'].describe()

count     9595.000000
mean      4317.476290
std       3298.325955
min        510.000000
25%       2152.000000
50%       3890.000000
75%       4708.500000
max      18484.000000
Name: gastos_ult_12m, dtype: float64

In [26]:
# gastos_ult_12m

etiquetas_g_u12 = ['menor_1k', '1k_a_4k', '4k_a_6k', '6k_a_8k', '8k_a_10k', 'mayor_10k']
rangos_g_u12 = [0, 999, 3999, 5999, 7999, 9999, 100000]
gastos_ult_12m_N = pd.cut(df_integrado['gastos_ult_12m'], 
                                bins=rangos_g_u12, 
                                labels=etiquetas_g_u12)

gastos_ult_12m_N.value_counts()

gastos_ult_12m
2k_a_4k      4872
4k_a_6k      3319
mayor_10k     663
6k_a_8k       296
8k_a_10k      287
menor_1k      158
Name: count, dtype: int64

In [28]:
df_integrado['limite_credito_tc'].describe()

count     9595.000000
mean      7260.316217
std       7046.150341
min       1438.300000
25%       2492.000000
50%       4199.000000
75%       9503.500000
max      34496.000000
Name: limite_credito_tc, dtype: float64

In [29]:
# limite_credito_tc

etiquetas_l_tc = ['menor_3k', '3k_a_5k', '5k_a_10k', 'mayor_10k']
rangos_l_tc = [0, 2999, 4999, 9999, 100000]
limite_credito_tc_N = pd.cut(df_integrado['limite_credito_tc'], 
                                bins=rangos_l_tc, 
                                labels=etiquetas_l_tc)

limite_credito_tc_N.value_counts()

limite_credito_tc
menor_3k     3432
mayor_10k    2239
5k_a_10k     2009
3k_a_5k      1915
Name: count, dtype: int64

In [30]:
df_integrado['operaciones_ult_12m'].describe()

count    9595.000000
mean       64.513809
std        23.190115
min        10.000000
25%        45.000000
50%        67.000000
75%        80.000000
max       138.000000
Name: operaciones_ult_12m, dtype: float64

In [31]:
# operaciones_ult_12m

etiquetas_o_u12 = ['menor_15', '15_a_30', '30_a_50', '50_a_75', '75_a_100', 'mayor_100']
rangos_o_u12 = [0, 14, 29, 49, 74, 99, 1000]
operaciones_ult_12m_N = pd.cut(df_integrado['operaciones_ult_12m'], 
                                bins=rangos_o_u12, 
                                labels=etiquetas_o_u12)

operaciones_ult_12m_N.value_counts()

operaciones_ult_12m
50_a_75      3227
75_a_100     2870
30_a_50      2319
mayor_100     594
15_a_30       564
menor_15       21
Name: count, dtype: int64

In [32]:
df_integrado.columns

Index(['estado_civil_N', 'estado_credito_N', 'id_cliente', 'edad',
       'importe_solicitado', 'duracion_credito', 'situacion_vivienda',
       'ingresos', 'objetivo_credito', 'pct_ingreso', 'estado_credito',
       'falta_pago', 'regla_pct_ingresos', 'regla_2', 'antiguedad_cliente',
       'estado_civil', 'estado_cliente', 'gastos_ult_12m', 'genero',
       'limite_credito_tc', 'nivel_educativo', 'operaciones_ult_12m',
       'personas_a_cargo'],
      dtype='object')

In [33]:
col_eliminar_final = [
              'edad',
              'regla_2',
              'antiguedad_cliente', 
              'ingresos',
              'pct_ingreso', 
              'regla_pct_ingresos',
              'gastos_ult_12m', 
              'limite_credito_tc', 
              'operaciones_ult_12m',
              'id_cliente']

df_integrado.drop(col_eliminar_final, inplace=True, axis=1)


df_final = pd.concat([operaciones_ult_12m_N, gastos_ult_12m_N, limite_credito_tc_N, antiguedad_cliente_N, ingresos_N, pct_ingreso_N, edad_N, df_integrado], axis=1)
df_final.head(5)


,operaciones_ult_12m,gastos_ult_12m,limite_credito_tc,antiguedad_cliente,ingresos,pct_ingreso,edad,estado_civil_N,estado_credito_N,importe_solicitado,duracion_credito,situacion_vivienda,objetivo_credito,estado_credito,falta_pago,estado_civil,estado_cliente,genero,nivel_educativo,personas_a_cargo
0,mayor_100,mayor_10k,3k_a_5k,2y_a_4y,50k_a_100k,hasta_20,menor_25,C,C,11000,3,HIPOTECA,INVERSIONES,0,N,CASADO,ACTIVO,F,SECUNDARIO_COMPLETO,3.0
1,15_a_30,menor_1k,3k_a_5k,2y_a_4y,20k_a_50k,hasta_20,menor_25,S,P,1500,2,ALQUILER,MEJORAS_HOGAR,1,Y,SOLTERO,PASIVO,M,DESCONOCIDO,0.0
2,15_a_30,2k_a_4k,5k_a_10k,2y_a_4y,20k_a_50k,20_a_40,menor_25,D,C,10000,2,OTROS,EDUCACIÓN,0,Y,DIVORCIADO,PASIVO,M,POSGRADO_COMPLETO,3.0
3,75_a_100,2k_a_4k,menor_3k,menor_2y,20k_a_50k,20_a_40,25_a_30,S,C,6000,4,ALQUILER,INVERSIONES,0,N,SOLTERO,ACTIVO,F,UNIVERSITARIO_INCOMPLETO,2.0
4,50_a_75,4k_a_6k,3k_a_5k,2y_a_4y,50k_a_100k,20_a_40,25_a_30,C,C,10000,2,HIPOTECA,EDUCACIÓN,0,N,CASADO,ACTIVO,F,UNIVERSITARIO_INCOMPLETO,2.0


In [34]:
col_eliminar_final_N = [
              'estado_credito',
              'estado_civil',]

df_final.drop(col_eliminar_final_N, inplace=True, axis=1)

Exportación de metadatos

In [35]:
import dtale as dt

informe = dt.show(df_final)
informe.open_browser()

Exportación del dataset

In [36]:
df_final.to_csv("../../data/final/datos_finales.csv", sep=';', index=False)

2023-12-03 16:48:25,983 - ERROR    - Exception occurred while processing request: Cannot setitem on a Categorical with a new category (Missing), set the categories first
Traceback (most recent call last):
  File "c:\Users\shaq4\anaconda3\envs\viu-13mbid\Lib\site-packages\dtale\views.py", line 119, in _handle_exceptions
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\shaq4\anaconda3\envs\viu-13mbid\Lib\site-packages\dtale\views.py", line 3102, in get_column_analysis
    return jsonify(**analysis.build())
                     ^^^^^^^^^^^^^^^^
  File "c:\Users\shaq4\anaconda3\envs\viu-13mbid\Lib\site-packages\dtale\column_analysis.py", line 142, in build
    return_data, code = self.analysis.build(self)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\shaq4\anaconda3\envs\viu-13mbid\Lib\site-packages\dtale\column_analysis.py", line 697, in build
    parent.data[parent.selected_col].fillna("Missing"), self.cleaners
    ^^^^^^^^^^^^^^^^^^